In [2]:
#importing libraries
import pandas as pd
import numpy as np

In [3]:
#reading data stored in local machine (available online)
reviews = pd.read_csv("reviews_detailed.csv")
calendar = pd.read_csv("calendar_detailed.csv")
listings = pd.read_csv("listings_detailed.csv")
# neighbourhoods = pd.read_csv("neighbourhoods.csv")

In [4]:
#cleaning the listings table by removing unwanted columns
listings.drop(["scrape_id","last_scraped","source","price", "bathrooms",'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'calendar_last_scraped','license','reviews_per_month'], axis = 1, inplace = True)

In [6]:
#creating new table with reviewers data. This includes reviewer name and id
reviewers = reviews[["reviewer_id","reviewer_name"]]
reviewers.drop_duplicates(subset=["reviewer_id"], inplace=True)
reviewers.reset_index(inplace=True)
reviewers.drop(columns=["index"], inplace=True)

/var/folders/jd/x6l823c51xl7ytjl3vh8jw1r0000gn/T/ipykernel_57403/3775423553.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewers.drop_duplicates(subset=["reviewer_id"], inplace=True)
/var/folders/jd/x6l823c51xl7ytjl3vh8jw1r0000gn/T/ipykernel_57403/3775423553.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewers.drop(columns=["index"], inplace=True)


In [7]:
#Creating a table property reviews that has all the properties and it's review scores
property_reviews = listings[["id",'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value']]

listings.drop(['number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value'], axis = 1, inplace = True)

In [8]:
#Creating a table that has host details stored
hosts = listings[['id', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified']]

listings.drop(['host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url','host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms'], axis = 1, inplace = True)

In [9]:
#Creating a neighbourhood table that stores details of neighbourhood
neighbourhood= listings[["neighbourhood_cleansed"]]
listings.drop(['neighbourhood','neighbourhood_group_cleansed'], axis=1, inplace = True)

### The available tables now are -
1. reviews
2. calendar_detailed
3. listings
4. host_detailed
5. neighbourhood_detailed
6. property_reviews

In [10]:
reviews.rename(columns={"id":"review_id"}, inplace = True)
reviews.drop(columns=["reviewer_name"], inplace=True)
reviews

,listing_id,review_id,date,reviewer_id,comments
0,3781,37776825,2015-07-10,36059247,The apartment was as advertised and Frank was ...
1,3781,41842494,2015-08-09,10459388,It was a pleasure to stay at Frank's place. Th...
2,3781,45282151,2015-09-01,12264652,The apartment description is entirely faithful...
3,3781,49022647,2015-09-30,41426327,Thoroughly enjoyed my time at Frank's home. Ha...
4,3781,52503327,2015-10-30,15151513,Great value for the money! This location has e...
...,...,...,...,...,...
174507,893975999133947853,898547534751030603,2023-05-24,516208792,"Very spacious, clean and comfortable. Highly r..."
174508,893975999133947853,908681741437409696,2023-06-07,169155474,thanks for all the helpful information about g...
174509,896170185862100440,918129405470822974,2023-06-20,14805192,Eddie is a wonderful host—he was super respons...
174510,896426215566639130,914465954641543171,2023-06-15,23667572,Cozy place!


In [11]:
calendar

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,3781,2023-06-22,f,$120.00,$120.00,32.0,1125.0
1,3781,2023-06-23,f,$120.00,$120.00,32.0,1125.0
2,3781,2023-06-24,f,$120.00,$120.00,32.0,1125.0
3,3781,2023-06-25,f,$120.00,$120.00,32.0,1125.0
4,3781,2023-06-26,f,$120.00,$120.00,32.0,1125.0
...,...,...,...,...,...,...,...
1450140,918690415212065897,2024-06-16,t,"$1,045.00",$941.00,1.0,365.0
1450141,918690415212065897,2024-06-17,t,"$1,039.00",$935.00,1.0,365.0
1450142,918690415212065897,2024-06-18,t,"$1,185.00","$1,067.00",1.0,365.0
1450143,918690415212065897,2024-06-19,t,"$1,207.00","$1,086.00",1.0,365.0


In [13]:
hosts

,id,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_verifications,host_has_profile_pic,host_identity_verified
0,53733454,107434423,https://www.airbnb.com/users/show/107434423,Blueground,2016-12-16,"New York, NY","We’re Blueground, a global proptech company wi...",within an hour,100%,97%,f,https://a0.muscache.com/im/pictures/user/d0ad9...,https://a0.muscache.com/im/pictures/user/d0ad9...,"['email', 'phone', 'work_email']",t,t
1,48001906,9410008,https://www.airbnb.com/users/show/9410008,Jason,2013-10-13,"Boston, MA",I live in the Back Bay and I'm happy to accomm...,within an hour,100%,100%,t,https://a0.muscache.com/im/users/9410008/profi...,https://a0.muscache.com/im/users/9410008/profi...,"['email', 'phone']",t,f
2,579003720446743556,814298,https://www.airbnb.com/users/show/814298,Thatch,2011-07-13,"Boston, MA",Thatch is an ever-evolving hospitality managem...,within an hour,99%,99%,NaN,https://a0.muscache.com/im/pictures/user/3cfc4...,https://a0.muscache.com/im/pictures/user/3cfc4...,"['email', 'phone']",t,t
3,582613302102919663,10647949,https://www.airbnb.com/users/show/10647949,Tom,2013-12-16,"Boston, MA","Enjoys traveling, good food, and outdoor sports",within an hour,100%,20%,f,https://a0.muscache.com/im/users/10647949/prof...,https://a0.muscache.com/im/users/10647949/prof...,"['email', 'phone']",t,t
4,820838307213765478,297860058,https://www.airbnb.com/users/show/297860058,Ferran,2019-09-25,NaN,"Ready, set, rent! Discover a rental experience...",within a few hours,94%,58%,NaN,https://a0.muscache.com/im/pictures/user/8fe7b...,https://a0.muscache.com/im/pictures/user/8fe7b...,"['email', 'phone']",t,t
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3968,740393253260304866,297860058,https://www.airbnb.com/users/show/297860058,Ferran,2019-09-25,NaN,"Ready, set, rent! Discover a rental experience...",within a few hours,94%,58%,NaN,https://a0.muscache.com/im/pictures/user/8fe7b...,https://a0.muscache.com/im/pictures/user/8fe7b...,"['email', 'phone']",t,t
3969,740400290363696561,346249638,https://www.airbnb.com/users/show/346249638,Jennifer,2020-05-14,"Boston, MA",NaN,within a few hours,100%,75%,f,https://a0.muscache.com/im/pictures/user/e0d70...,https://a0.muscache.com/im/pictures/user/e0d70...,"['email', 'phone']",t,f
3970,741745079064862710,107434423,https://www.airbnb.com/users/show/107434423,Blueground,2016-12-16,"New York, NY","We’re Blueground, a global proptech company wi...",within an hour,100%,97%,f,https://a0.muscache.com/im/pictures/user/d0ad9...,https://a0.muscache.com/im/pictures/user/d0ad9...,"['email', 'phone', 'work_email']",t,t
3971,741745368954817653,107434423,https://www.airbnb.com/users/show/107434423,Blueground,2016-12-16,"New York, NY","We’re Blueground, a global proptech company wi...",within an hour,100%,97%,f,https://a0.muscache.com/im/pictures/user/d0ad9...,https://a0.muscache.com/im/pictures/user/d0ad9...,"['email', 'phone', 'work_email']",t,t


In [15]:
property_reviews

,id,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,53733454,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48001906,75,26,1,2021-02-22,2023-05-30,4.97,5.00,4.96,5.00,4.99,4.97,4.79
2,579003720446743556,27,24,1,2022-05-29,2023-06-17,4.48,4.67,4.44,4.74,4.56,4.67,4.44
3,582613302102919663,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,820838307213765478,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3968,740393253260304866,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3969,740400290363696561,1,1,0,2023-04-18,2023-04-18,5.00,5.00,5.00,5.00,5.00,5.00,5.00
3970,741745079064862710,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3971,741745368954817653,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
num = 0
mapping = {}
for location in listings.neighbourhood_cleansed.unique():
    num+=1
    mapping[location] = num

num=0
for location in listings.neighbourhood_cleansed.unique():
    num+=1
    listings.replace({location:num}, inplace=True)

In [18]:
neighbourhood.drop_duplicates(inplace=True)
neighbourhood.reset_index(drop=True, inplace=True)

df = pd.DataFrame.from_dict(mapping, orient="index").reset_index()
df.rename(columns={"index":"neighbourhood_cleansed",0:"neighbourhood_id"}, inplace=True)
neighbourhood = pd.merge(neighbourhood,df, on="neighbourhood_cleansed")

/var/folders/jd/x6l823c51xl7ytjl3vh8jw1r0000gn/T/ipykernel_57403/12498285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neighbourhood.drop_duplicates(inplace=True)


In [19]:
neighbourhood

,neighbourhood_cleansed,neighbourhood_id
0,South Boston Waterfront,1
1,North End,2
2,South Boston,3
3,East Boston,4
4,Allston,5
5,Downtown,6
6,Roxbury,7
7,Roslindale,8
8,Mattapan,9
9,Charlestown,10


In [20]:
listings.rename(columns={"neighbourhood_cleansed":"neighbourhood_id"},inplace=True)

In [21]:
listings

,id,listing_url,name,description,neighborhood_overview,picture_url,neighbourhood_id,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,instant_bookable
0,53733454,https://www.airbnb.com/rooms/53733454,Rental unit in Boston · 1 bedroom · 1 bed · 1 ...,Show up and start living from day one in Bosto...,This furnished apartment is located in Seaport...,https://a0.muscache.com/pictures/prohost-api/H...,1,42.351397,-71.043191,Entire rental unit,Entire home/apt,2,1 bath,1.0,1.0,"[""Elevator"", ""Hair dryer"", ""Smoke alarm"", ""Hot...",f
1,48001906,https://www.airbnb.com/rooms/48001906,Rental unit in Boston · ★4.97 · 3 bedrooms · 3...,Newly renovated 3 Bedroom | 1 Bath ground-leve...,The North End is Boston's oldest residential n...,https://a0.muscache.com/pictures/853ccf49-8c81...,2,42.367160,-71.054990,Entire rental unit,Entire home/apt,6,1 bath,3.0,3.0,"[""Body soap"", ""Cooking basics"", ""Bathtub"", ""Ha...",t
2,579003720446743556,https://www.airbnb.com/rooms/579003720446743556,Rental unit in Boston · ★4.48 · 1 bedroom · 1 ...,Thatch Quarters™ on Broadway is located in the...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,3,42.343160,-71.055700,Private room in rental unit,Private room,2,2 shared baths,1.0,1.0,"[""Washer"", ""Smoke alarm"", ""Self check-in"", ""Wi...",t
3,582613302102919663,https://www.airbnb.com/rooms/582613302102919663,Condo in Boston · 2 bedrooms · 2 beds · 2 baths,Bright and spacious recently renovated condo i...,"Historic East Boston with vibrating nightlife,...",https://a0.muscache.com/pictures/miso/Hosting-...,4,42.379050,-71.030990,Entire condo,Entire home/apt,4,2 baths,2.0,2.0,"[""Body soap"", ""Marina view"", ""Free washer \u20...",f
4,820838307213765478,https://www.airbnb.com/rooms/820838307213765478,Rental unit in Boston · 3 bedrooms · 1 bed · 0...,This 99 square footage room on Boston's Allsto...,NaN,https://a0.muscache.com/pictures/prohost-api/H...,5,42.348458,-71.135604,Private room in rental unit,Private room,1,0 shared baths,NaN,1.0,"[""Cooking basics"", ""Free street parking"", ""Loc...",f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3968,740393253260304866,https://www.airbnb.com/rooms/740393253260304866,Rental unit in Boston · 5 bedrooms · 1 bed · 0...,This 112 square footage room on Boston's spaci...,NaN,https://a0.muscache.com/pictures/prohost-api/H...,11,42.340680,-71.146230,Private room in rental unit,Private room,1,0 shared baths,NaN,1.0,"[""Cooking basics"", ""Lockbox"", ""TV"", ""Heating"",...",f
3969,740400290363696561,https://www.airbnb.com/rooms/740400290363696561,Condo in Boston · 2 bedrooms · 2 beds · 2 baths,This beautiful home in the central and histori...,"The South End is in central Boston, adjacent t...",https://a0.muscache.com/pictures/6679d540-9cda...,15,42.344061,-71.075566,Entire condo,Entire home/apt,4,2 baths,2.0,2.0,"[""Body soap"", ""Free washer \u2013 In unit"", ""C...",f
3970,741745079064862710,https://www.airbnb.com/rooms/741745079064862710,Rental unit in Boston · 3 bedrooms · 3 beds · ...,Show up and start living from day one in Bosto...,Set yourself up in this furnished apartment in...,https://a0.muscache.com/pictures/prohost-api/H...,12,42.319087,-71.060710,Entire rental unit,Entire home/apt,6,1 bath,3.0,3.0,"[""Hair dryer"", ""Smoke alarm"", ""Hot water"", ""Se...",f
3971,741745368954817653,https://www.airbnb.com/rooms/741745368954817653,Rental unit in Boston · 1 bedroom · 1 bed · 1 ...,Show up and start living from day one in Bosto...,This furnished apartment is located in Brighto...,https://a0.muscache.com/pictures/prohost-api/H...,11,42.342301,-71.146945,Entire rental unit,Entire home/apt,2,1 bath,1.0,1.0,"[""Hair dryer"", ""Smoke alarm"", ""Hot water"", ""Se...",f


In [35]:
import os

directory = "./processed_csv_files"

# Check if the directory exists
if not os.path.exists(directory):
    # If it doesn't exist, create it
    os.makedirs(directory)
    
    
list_of_tables = [reviews,calendar,listings,property_reviews,neighbourhood,hosts]
list_of_tables_names = ['reviews','calendar','listings','property_reviews','neighbourhood','hosts']

for table,table_name in zip(list_of_tables,list_of_tables_names):
    table.to_csv(f"./processed_csv_files/{table_name}.csv")